Script to Delete Retired Units that are invalid
Update Unit Status from Held to issue, as well as the original unit counts


For context, all the uploaded units that belong to this orgUid are all CDM data. Therefore, we dont need to run another script with v1/projects API to match whose unit the registry belong too

In [ ]:
# Reproduce the code
# Get Units
# Get all units for a given organization
# {{hostURL}}/v1/units?page=1&limit=100&orgUid={{orgUid}}

# Loop through all pages and store in dataframe
# Only store it if "unitStatus" is "retired"
# Just nice that all the "retired" units all belong to CDM. No need to have another checking whose unit belong to what registry. Otherwise, proeject level APi checking is needed

In [1]:
import pandas as pd
import requests
import json


In [ ]:


# THIS SCRIPT IS TO RUN ONCE ONLY, THEN SAVED THE UNITS TO CSV FILE. FOR FURTHER ETL PROCESSING, USE THE CSV FILE SO NO MORE API CALLS ARE NEEDED


# Define the base URL and organization UID
base_url = "http://localhost:31310/v1/units"
org_uid = "b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985"

# Initialize variables
page = 1
limit = 100
all_units = []

# Loop through all pages
while True:
    # Construct the URL with pagination and organization UID
    url = f"{base_url}?page={page}&limit={limit}&orgUid={org_uid}"
    
    # Make the GET request
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to fetch data: {response.status_code}")
        break
    
    # Parse the response JSON
    data = response.json()
    
    # Extract units from the response
    units = data.get("data", [])
    
    # Filter for units with "unitStatus" == "Retired"
    retired_units = [unit for unit in units if unit.get("unitStatus") == "Retired"]
    all_units.extend(retired_units)
    
    # Check if there are more pages
    if page >= data.get("pageCount", 0):
        break
    
    # Increment the page number
    page += 1

# Convert the list of retired units to a DataFrame
df_retired_units = pd.DataFrame(all_units)

# Save the DataFrame to a CSV file for further analysis
df_retired_units.to_csv("retired_units.csv", index=False)

print(f"Fetched {len(df_retired_units)} retired units.")

Fetched 27747 retired units.


0        3408858e-5603-4daa-bcfc-825940b7eb23
1        a77fc93e-f2a2-446a-a87c-5eebae8909a0
2        7d755114-c7cd-4294-8786-50a0acbff1c5
3        b28eff03-5e40-4b32-a8cd-d62844c0b391
4        59ceac77-20d8-43f4-bb5c-7f1d16315d32
                         ...                 
27742    21da8ddd-d12a-44f8-86d2-fad2b78b6fa6
27743    c573dece-f6f9-41c8-9f7c-1c64da95153b
27744    dbd6657c-1f11-46cf-bedd-52b7e3a2cd69
27745    af84c43c-6eba-4218-96c7-042996e27c5d
27746    85a1e745-55b1-44e1-ac75-37abf27d9b59
Name: warehouseUnitId, Length: 27747, dtype: object

In [13]:
df_retired_units = pd.read_csv("CDMRetiredUnits24Apr.csv")

retired_CDM_ids = df_retired_units['warehouseUnitId']



def delete_retired_units(df_retired_units, base_url, start_index=0, end_index=None):
    """
    Deletes retired units based on their warehouseUnitId.

    Parameters:
        df_retired_units (DataFrame): DataFrame containing retired units.
        base_url (str): The base URL for the DELETE API.
        start_index (int): The starting index for deletion.
        end_index (int): The ending index for deletion. If None, deletes until the end of the DataFrame.
    """
    # Get the list of warehouseUnitIds
    retired_CDM_ids = df_retired_units['warehouseUnitId'][start_index:end_index]

    # Loop through each retired unit ID and send a DELETE request
    for idx,unit_id in enumerate(retired_CDM_ids):
        # Construct the request body
        request_body = {
            "warehouseUnitId": unit_id
        }
        
        # Send the DELETE request
        response = requests.delete(base_url, json=request_body)
        
        # Check the response status
        if response.status_code == 200:
            print(f"Successfully deleted unit entry number {start_index+idx} with ID: {unit_id}")
        else:
            print(f"Failed to delete unit with ID: {unit_id}. Status code: {response.status_code}. Message: {response.text}")

# Example usage
base_url = "http://localhost:31310/v1/units"
start_index = 27500  # Specify the starting index
end_index = 28000  # Specify the ending index (or None to delete all remaining units)

delete_retired_units(df_retired_units, base_url, start_index, end_index)

Successfully deleted unit entry number 27500 with ID: d76e29d3-2426-4b66-9777-8aa76c6054c2
Successfully deleted unit entry number 27501 with ID: ab2f8447-7ae2-41da-80b6-4445f2f14d4c
Successfully deleted unit entry number 27502 with ID: d1217f63-0839-4958-9e13-e51095a05a09
Successfully deleted unit entry number 27503 with ID: 1fe169ca-3db8-4c0f-8101-92f31e3a0dd1
Successfully deleted unit entry number 27504 with ID: c5219d5f-18c7-48f4-bd2e-9c4a64a90552
Successfully deleted unit entry number 27505 with ID: de2beefb-3849-4709-846b-189b783bade9
Successfully deleted unit entry number 27506 with ID: e0027f62-c12e-4776-ac08-3d738e37b195
Successfully deleted unit entry number 27507 with ID: ae8fd9a0-97f7-4488-9ffc-8f4f9f7db5af
Successfully deleted unit entry number 27508 with ID: bb1b12b8-be1d-4863-9579-4fe571e7f4dc
Successfully deleted unit entry number 27509 with ID: 4f5e417c-27da-4491-b9bb-5ba0d3c80b9e
Successfully deleted unit entry number 27510 with ID: 654a3694-8321-4123-ae77-2379697efcd1